In [1]:
import numpy as np
import cv2
import dlib

In [2]:
import sys
sys.path.append("/home/yonathan/Documents/projects/faceSwap")
import faceSwap

In [3]:
    def landmark_detection(image, returnImage=False):
        """Generate facial landmark points of a give image.

        Parameters
        ----------
        image : str
            image file path.

        Returns
        -------
        landmark_points : list
            return a list of tuple integer of the landmark points.
        img : numpy array
            return an image where the landmark points draw on the the image.

        """

        # laod an image then convert it to grey scale
        img = cv2.imread(image)
        img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # detect the face then find the landmarks
        detector = dlib.get_frontal_face_detector()
        predictor = dlib.shape_predictor(
            'shape_predictor_68_face_landmarks.dat')

        faces = detector(img_gray)
        faces_landmark_points = []
        for face in faces:
            landmarks = predictor(img_gray, face)
            landmark_points = []
            for n in range(68):
                x = landmarks.part(n).x
                y = landmarks.part(n).y
                landmark_points.append((int(x), int(y)))

                cv2.circle(img, (x, y), 3, (0, 0, 255), -1)

            faces_landmark_points.append(landmark_points)

        if returnImage:
            return faces_landmark_points, img
        else:
            return faces_landmark_points

In [4]:
image = "/home/yonathan/Documents/projects/faceSwap/images/original_images/anchorman.jpg"

In [5]:
faces_landmark_points, mod_img = landmark_detection(image, True)

In [6]:
faces = len(faces_landmark_points)

In [17]:
def choose_largest_face(faces_landmark_points, img):

    size = 0
    faces = np.array(faces_landmark_points).shape[0]
    for face in range(faces):
        boundingRect = cv2.boundingRect(np.array(faces_landmark_points[face]))

        face_size = boundingRect[2] * boundingRect[3]
        if face_size > size:
            size = face_size
            larger_face_index = face

    largest_face_landmark_points = faces_landmark_points[larger_face_index]
    return largest_face_landmark_points

In [18]:
landmark_points = faces_landmark_points[0]
np.array(landmark_points).shape

(68, 2)

In [19]:
boundingRect = cv2.boundingRect(np.array(landmark_points))
boundingRect

(253, 318, 162, 158)

In [20]:
size = (boundingRect[2] - boundingRect[0]) * (boundingRect[3] - boundingRect[1])
size

14560

In [21]:
img = cv2.imread(image)
index = choose_largest_face(faces_landmark_points, img)
cv2.imshow("Image111", img)
cv2.imshow("Mod", mod_img)
cv2.waitKey(0)
cv2.destroyAllWindows()